In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# plot the mse of the function f(x): x*\theta = 0
v = np.linspace(-1, 1, 100)
mse = v**2
plt.plot(v, mse)
plt.plot(v, v*2)
#use latex for the labels
plt.rcParams['text.usetex'] = True

plt.xlabel(r'$\theta$')
plt.ylabel(r'$E_{MSE}$')
plt.show()

In [ ]:
from matplotlib import pyplot as plt
import tensorflow as tf
keras = tf.keras
from keras import losses, layers, models, metrics, optimizers
from matplotlib import pyplot as plt


data = tf.keras.datasets.mnist.load_data(
    path='mnist.npz'
)

train, test = data

train_data, train_labels = train
test_data, test_labels = test

train_labels = tf.one_hot(train_labels, 10)
test_labels = tf.one_hot(test_labels, 10)

plt.imshow(train_data[0], cmap='gray')
plt.show()


import tensorflow as tf
keras = tf.keras
from keras import losses, layers, models, metrics

network = models.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Normalization(),
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

network2 = models.Sequential([
    #layers.Normalization(input_shape=(28, 28, 1)),
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (1,1), activation='relu'),
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

network.compile(loss = losses.CategoricalCrossentropy(), metrics=[metrics.CategoricalAccuracy()], optimizer=optimizers.SGD(learning_rate=0.001))

training = network.fit(train_data, train_labels, epochs=20, batch_size=128, validation_data=(test_data, test_labels))


In [ ]:
# show loss and accuracy in twin x plots
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.plot(training.history['loss'], label='loss', color='red')
ax1.plot(training.history['val_loss'], label='val_loss', color='orange')
ax1.set_xlabel('epoch')
ax1.set_ylabel('loss')
ax1.legend()
ax2.plot(training.history['categorical_accuracy'], label='accuracy', color='blue')	
ax2.plot(training.history['val_categorical_accuracy'], label='val_accuracy', color='green')
ax2.set_ylabel('accuracy')
ax2.legend()
plt.show()

In [ ]:
network.summary()


In [ ]:
import skimage
from support.bounding_boxes import create_bounding_boxes, draw_bounding_boxes

img = skimage.io.imread('images/Numbers.jpg', as_gray=True)
# resize image
img = skimage.transform.resize(img, (200, 300))

# increase contrast
#img = skimage.exposure.rescale_intensity(img, in_range=(0.2, 0.8))

plt.imshow(img, cmap='gray')
plt.show()

# do canny edge
edges = skimage.feature.canny(img, sigma=1)
plt.imshow(edges, cmap='gray')
plt.show()

# erosion
edges = skimage.morphology.binary_dilation(edges, footprint=np.ones((5,5)))
plt.imshow(edges, cmap='gray')
plt.show()

#label image
labels = skimage.measure.label(edges)

# create bounding boxes
rectangles = create_bounding_boxes(labels, square = True)
plt.imshow(img, cmap='gray')
draw_bounding_boxes(rectangles)
plt.show()
assigned_classes = []

# create bounding boxes with score
for rect in rectangles:
    x, y, w, h = rect
    # extract patch for rectangle
    patch = img[y:y+h, x:x+w]
    # resize patch to required size
    patch = skimage.transform.resize(patch, (28, 28))
    # invert colors
    patch = 1. - patch
    patch = np.where(patch < 0.5, 0, patch * 2 - 0.5)
    # get the network output for the patch
    output = network1(tf.expand_dims(patch * 255, 0))
    # get the predicted class
    label = np.argmax(output[0])
    assigned_classes.append(label)

plt.imshow(img, cmap='gray')
draw_bounding_boxes(rectangles, assigned_classes)
plt.show()

In [ ]:
# plot random samples of two classes on an x y plot
import numpy as np
import matplotlib.pyplot as plt

# create random samples
samples = np.random.randn(100, 2)
# create labels
labels = samples[:,0] > samples[:,1]
labels[60:62] = labels[60:62] == False
# plot samples
plt.scatter(samples[:, 0], samples[:, 1], c=labels)
plt.show()

# create a network that can separate the two classes
network = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(2,)),
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

network.compile(loss = losses.BinaryCrossentropy(), metrics=[metrics.BinaryAccuracy()], optimizer=optimizers.SGD(learning_rate=0.05))

training = network.fit(samples, labels, epochs=1000, batch_size=128)

# plot the class boundary as line
x = np.linspace(-3,3, 50)
y = np.linspace(-3,3, 50)
X, Y = np.meshgrid(x, y)

Z = network.predict(np.stack([X.flatten(), Y.flatten()], axis=-1))
Z = Z.reshape((50, 50))
print(Z.shape)
plt.contourf(X, Y, Z, levels=1)
plt.scatter(samples[:, 0], samples[:, 1], c=labels)
plt.show()




In [ ]:
from matplotlib import pyplot as plt
import tensorflow as tf
keras = tf.keras
from keras import losses, layers, models, metrics, optimizers, callbacks
from matplotlib import pyplot as plt
import os
import skimage
import numpy as np

# load the data from images/training_png
data = []
labels = []
classes = []
for i, folder in enumerate(os.listdir('images/training_png')):
    classes.append(folder)
    for filename in os.listdir(f'images/training_png/{folder}'):
        img = skimage.io.imread(f'images/training_png/{folder}/{filename}')
        img = skimage.transform.resize(img, (64,64))
        data.append(img)
        labels.append(i)

data = np.stack(data)
labels = np.array(labels)
splits = np.random.permutation(len(data))
data = data[splits]
labels = labels[splits]
labels = tf.one_hot(labels, len(classes))
#split into test and train data
data_train = data[:int(len(data) * 0.8)]
labels_train = labels[:int(len(data) * 0.8)]
data_test = data[int(len(data) * 0.8):]
labels_test = labels[int(len(data) * 0.8):]

# create a network that can separate the two classes

network = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(None, None, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Lambda(tf.reduce_mean, arguments={'axis': [1, 2]}),
    layers.Flatten(),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(2, activation='softmax')
])
network.summary()
network.compile(loss = losses.MeanSquaredError(), metrics=[metrics.CategoricalAccuracy()], optimizer=optimizers.Adam())

# plot dataset as image matrix
fig, axes = plt.subplots(5, len(data_train) // 5 + 1, figsize=(len(data_train) // 5 + 3, 7))
# flatten axes
axes = axes.flatten()
for i, ax in enumerate(axes):
    if i < len(data_train):
        ax.imshow(data_train[i])
        ax.set_title(classes[tf.argmax(labels_train[i]).numpy()])
    ax.axis('off')
plt.show()

training = network.fit(data_train, labels_train, epochs=100, batch_size=128, validation_data=(data_test, labels_test), 
                       callbacks=[callbacks.EarlyStopping(patience=5, monitor='val_loss', restore_best_weights=True)])

In [ ]:
# show loss and accuracy in twin x plots
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.plot(training.history['loss'], label='loss', color='red')
ax1.plot(training.history['val_loss'], label='val_loss', color='orange')
ax1.set_xlabel('epoch')
ax1.set_ylabel('loss')
ax1.legend()
ax2.plot(training.history['categorical_accuracy'], label='accuracy', color='blue')	
ax2.plot(training.history['val_categorical_accuracy'], label='val_accuracy', color='green')
ax2.set_ylabel('accuracy')
ax2.legend()
plt.show()

In [ ]:
# add images containing random noise to test data
ood = np.random.rand(len(data_test), 64, 64, 3)
data_test = np.concatenate([data_test, ood])

# plot dataset as image matrix
fig, axes = plt.subplots(len(data_test) // 5 + 1, 5, figsize=(7, len(data_test) // 3 + 2))
# flatten axes
axes = axes.flatten()
pred = network.predict(data_test)

# calculate confidence as entropy
conf = 1 - tf.reduce_sum(-pred * tf.math.log(pred + 0.001), axis=-1)
for i, ax in enumerate(axes):
    if i < len(data_test):
        ax.imshow(data_test[i])
        
        ax.set_title(f'{classes[tf.argmax(pred[i]).numpy()]}: {conf[i]:.2f}')
    ax.axis('off')
plt.show()

In [ ]:
columns = 4
#Get outputs of each individual layer
outputs = [layer.output for layer in network.layers]
# create a model that outputs the layers
model = models.Model(inputs=network.inputs, outputs=outputs)
# get the output of the first layer
layer_output = model.predict(data_test[0:1])
# plot the output of the first layer
for layer in layer_output:
    if len(layer.shape) < 3:
        continue
    feature_maps = layer.shape[-1]
    fig, axes = plt.subplots(4, feature_maps //columns, figsize=( feature_maps//columns + 1, columns + 1))
    axes = axes.flatten()
    for i, ax in enumerate(axes):
        if i < feature_maps:
            ax.imshow(layer[0,:,:,i])
            ax.axis('off')
    plt.show()


In [ ]:
#Plot sigmoid function and its derivative
x = np.linspace(-10, 10, 100)
y = 1 / (1 + np.exp(-x))
y_prime = y * (1 - y)
plt.plot(x, y, label='sigmoid')
plt.plot(x, y_prime, label='sigmoid derivative')
plt.legend()
plt.show()
